In [2]:
import pennylane as qml
from pennylane import numpy as np
from qiskit.providers.fake_provider import *
from pennylane.transforms import mitigate_with_zne, richardson_extrapolate, fold_global
def configured_backend():
    return FakeManila()

def ansatz(params):
    qml.RX(params[0], wires=0)
    qml.RZ(params[1], wires=0)
def CA():
    qml.CZ(wires=(1, 0))
def U_b():
    qml.Hadamard(wires=0)
dev_noisy = qml.device("qiskit.remote", wires = 2, backend=configured_backend())
dev = qml.device("default.qubit", wires = 2, shots = 1000)

def circuit(params, part=None, j=-1):
    qml.Hadamard(1)
    if part == "Im":
        qml.PhaseShift(-np.pi/2, wires=1)

    ansatz(params)
    CA()
    U_b()

    if j != -1:
        qml.CZ(wires=[1, j])

    U_b()
    CA()

    qml.Hadamard(wires=1)

    return qml.expval(qml.PauliZ(wires=1))

# circuit = qml.QNode(circuit, dev)
circuit = qml.QNode(circuit, dev_noisy)
circuit = mitigate_with_zne(circuit, [1, 2, 3], fold_global, richardson_extrapolate)
def mu(params, j):
    mu_real = circuit(params, part="Re", j=j)
    mu_imag = circuit(params, part="Im", j=j)
    
    return mu_real + 1.0j * mu_imag
def cost_fun(params):
    mu_sum = abs(mu(params, 0))
    norm = abs(mu(params, -1))

    res = 0.5 - 0.5 * mu_sum / (1 * norm)

    return res
lr = 0.01
opt = qml.GradientDescentOptimizer(lr)
from IPython.display import clear_output
w = np.random.randn(2, 
requires_grad=True)
cost_history = []

for it in range(100):
    w, cost = opt.step_and_cost(cost_fun, w)
    clear_output(wait=True)
    print("Step {:3d}       Cost_L = {:9.7f}".format(it, cost), flush=True)
    cost_history.append(cost)

import matplotlib.pyplot as plt
plt.style.use("seaborn")
# plt.plot(np.log(cost_history), "g")
plt.plot(cost_history, "g")
plt.ylabel("Cost function")
plt.xlabel("Optimization steps")
plt.show()

Step   1       Cost_L = 0.4923462


KeyboardInterrupt: 

In [3]:
qml.about()

Name: PennyLane
Version: 0.33.1
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /Users/bigsad/Downloads/Algorithm-Research/Student-Hub/Indy-Ng/.venv/lib/python3.11/site-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, requests, rustworkx, scipy, semantic-version, toml, typing-extensions
Required-by: PennyLane-Lightning, PennyLane-qiskit

Platform info:           macOS-12.6-x86_64-i386-64bit
Python version:          3.11.6
Numpy version:           1.23.5
Scipy version:           1.10.1
Installed devices:
- default.gaussian (PennyLane-0.33.1)
- default.mixed (PennyLane-0.33.1)
- default.qubit (PennyLane-0.33.1)
- default.qubit.autograd (PennyLane-0.33.1)
- default.qubit.jax (PennyLane-0.33.1)
- default.qubit.legacy (PennyLane-0.33.1)
- default.qubit.tf (PennyLane-0.33.1)
- default.qubit.torch (Penn